# Face detection and recognition inference pipeline

The following example illustrates how to use the `facenet_pytorch` python package to perform face detection and recogition on an image dataset using an Inception Resnet V1 pretrained on the VGGFace2 dataset.

The following Pytorch methods are included:
* Datasets
* Dataloaders
* GPU/CPU processing

In [15]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os
from PIL import Image
workers = 0 if os.name == 'nt' else 4

#### Determine if an nvidia GPU is available

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module

Default params shown for illustration, but not needed. Note that, since MTCNN is a collection of neural nets and other code, the device must be passed in the following way to enable copying of objects when needed internally.

See `help(MTCNN)` for more details.

In [5]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Define Inception Resnet V1 module

Set classify=True for pretrained classifier. For this example, we will use the model to output embeddings/CNN features. Note that for inference, it is important to set the model to `eval` mode.

See `help(InceptionResnetV1)` for more details.

In [6]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

#### Define a dataset and data loader

We add the `idx_to_class` attribute to the dataset to enable easy recoding of label indices to identity names later one.

In [7]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('D:\\dev\\project\\pythonProject\\temp_img')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

FileNotFoundError: Couldn't find any class folder in D:\dev\project\pythonProject\temp_img.

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and detect faces and associated detection probabilities for each. The `MTCNN` forward method returns images cropped to the detected face, if a face was detected. By default only a single detected face is returned - to have `MTCNN` return all detected faces, set `keep_all=True` when creating the MTCNN object above.

To obtain bounding boxes rather than cropped face images, you can instead call the lower-level `mtcnn.detect()` function. See `help(mtcnn.detect)` for details.

In [32]:
aligned = []
names = []
folder_name = 'D:/dev/project/pythonProject/temp_img/'
for (root, dirs, files) in os.walk(folder_name, topdown = True):
    for path in files:
        img = Image.open(folder_name + path).convert('RGB')
        
        x_aligned, prob = mtcnn(img, return_prob=True)
        print(path, ' : ')
        if x_aligned is not None:
            print('Face detected with probability: {:8f}'.format(prob))
            aligned.append(x_aligned)
            names.append(path)
        else:
            print("bro wtf")

0977833497_34.jpg  : 
Face detected with probability: 0.785608
0977833497_35.jpg  : 
bro wtf


#### Calculate image embeddings

MTCNN will return images of faces all the same size, enabling easy batch processing with the Resnet recognition module. Here, since we only have a few images, we build a single batch and perform inference on it. 

For real datasets, code should be modified to control batch sizes being passed to the Resnet, particularly if being processed on a GPU. For repeated testing, it is best to separate face detection (using MTCNN) from embedding or classification (using InceptionResnetV1), as calculation of cropped faces or bounding boxes can then be performed a single time and detected faces saved for future use.

In [18]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

RuntimeError: stack expects a non-empty TensorList

#### Print distance matrix for classes

In [8]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))

                angelina_jolie  bradley_cooper  kate_siegel  paul_rudd  \
angelina_jolie        0.000000        1.344806     0.781201   1.425579   
bradley_cooper        1.344806        0.000000     1.256238   0.922126   
kate_siegel           0.781201        1.256238     0.000000   1.366423   
paul_rudd             1.425579        0.922126     1.366423   0.000000   
shea_whigham          1.448495        0.891145     1.416447   0.985438   

                shea_whigham  
angelina_jolie      1.448495  
bradley_cooper      0.891145  
kate_siegel         1.416447  
paul_rudd           0.985438  
shea_whigham        0.000000  
